In [4]:
from lib.NeuralNet import *
from lib.Optimizer import UpdateParamsAdam

In [5]:
X = np.random.rand(5000,100)
y = np.random.rand(5000,1)
W1,b1,W2,b2,W3,b3 = FitModel(X,y,n_iter=1000,batch_size=500,lr=1e-3,UpdateParams=UpdateParamsAdam,print_stat=True)

Epoch  0 Loss:  0.08511019551061502
Epoch  50 Loss:  0.05335146355609445
Epoch  100 Loss:  0.02773705549998517
Epoch  150 Loss:  0.014919916665629408
Epoch  200 Loss:  0.006831649232610667
Epoch  250 Loss:  0.006049210544023379
Epoch  300 Loss:  0.0038329320432061273
Epoch  350 Loss:  0.0038737859416404044
Epoch  400 Loss:  0.0028210897853134464
Epoch  450 Loss:  0.0022798109696045422
Epoch  500 Loss:  0.001935071795944262
Epoch  550 Loss:  0.0017556108215542718
Epoch  600 Loss:  0.0011988342246808698
Epoch  650 Loss:  0.0015841795939574084
Epoch  700 Loss:  0.00137888069376637
Epoch  750 Loss:  0.0010030917594360338
Epoch  800 Loss:  0.0017415181169827007
Epoch  850 Loss:  0.0013476953643571652
Epoch  900 Loss:  0.0009426442551701863
Epoch  950 Loss:  0.001090098154664576
